# This notebook is to evaluate network performance on test shots

In [ ]:
# imports
from scipy.io import loadmat
import os
from os.path import join
import matplotlib.pyplot as plt
CM = 'inferno'
CM_R = 'inferno_r'
plt.rcParams['image.cmap'] = CM_R

In [ ]:
import numpy as np

# random cell to show the activation function
def swish_act(x, β):
    return x * (1 / (1 + np.exp(-β * x)))

xs = np.linspace(-4, 4, 1000)
βs = [0.1, 0.5, 1, 2, 5]
plt.figure(figsize=(6, 4))
for β in βs:
    plt.plot(xs, swish_act(xs, β), label=f'β={β}', linewidth=2)
plt.title('Swish Activation Function')
plt.xlabel('x')
plt.ylabel('Swish(x)')
plt.legend()
plt.savefig('swish_activation.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# let's first see what shots we have
SHOTS_DIR = 'test_shots'
SHOTS_NET_DIR = 'test_shots_net'
os.makedirs(SHOTS_NET_DIR, exist_ok=True)

# list all the files inside
SHOT_FILES = os.listdir(SHOTS_DIR)
SHOT_NAMES = [f.split('.')[0] for f in SHOT_FILES if f.endswith('.mat')]
print(f'Found {len(SHOT_NAMES)} shots: {SHOT_NAMES}')

# speed up, keep only the first 2
SHOT_NAMES, SHOT_FILES = SHOT_NAMES[:2], SHOT_FILES[:2]
print(f'Using {len(SHOT_NAMES)} shots: {SHOT_NAMES}')

In [ ]:
# run matlab inference on each shot
for s, sn in zip(SHOT_FILES, SHOT_NAMES):
    print(f'Running matlab inference on {s}...')
    command_linux = f'matlab -nosplash -nodesktop -r "run_net_on_shot({sn}, \'{SHOTS_NET_DIR}\'); exit;"'
    command_mac = f'/Applications/MATLAB_R2024b.app/bin/matlab -nosplash -nodesktop -r "run_net_on_shot({sn}, \'{SHOTS_NET_DIR}\'); exit;"'
    # os.system(command_linux)
    os.system(command_mac)

In [ ]:
# plotting time baby
import numpy as np
from utils import plot_vessel

g = loadmat('tcv_params/grid.mat')
r, z = g['r'].flatten(), g['z'].flatten()
rr, zz = np.meshgrid(r, z)
rr, zz = rr.flatten(), zz.flatten()

for s, sn in zip(SHOT_FILES, SHOT_NAMES):
    # load
    gt = loadmat(join(SHOTS_DIR, s))
    net = loadmat(join(SHOTS_NET_DIR, f'{sn}_net.mat'))

    ri = 1000

    gFx = gt['Fx']
    gBr = gt['Br']
    gBz = gt['Bz']

    nFx = net['Fx']
    nBr = net['Br']
    nBz = net['Bz']

    t = gt['t'].flatten()

    # range
    mFx, MFx = np.min(gFx, axis=(0, 1)), np.max(gFx, axis=(0, 1))
    mBr, MBr = np.min(gBr), np.max(gBr) 
    mBz, MBz = np.min(gBz), np.max(gBz) 
    rFx = MFx - mFx
    rBr = MBr - mBr
    rBz = MBz - mBz
    # percentage error 
    eFx = 100*np.abs(gFx - nFx) / rFx
    eBr = 100*np.abs(gBr - nBr) / rBr
    eBz = 100*np.abs(gBz - nBz) / rBz

    max_perc_err = 5.0  # max percentage error for colorbar

    plt.figure(figsize=(15, 15))
    plt.subplot(3, 3, 1)
    plot_vessel()
    sc1 = plt.scatter(rr, zz, c=gFx[..., ri], vmin=mFx[ri], vmax=MFx[ri], cmap=CM_R) 
    plt.title('LIUQUE Fx')
    plt.axis('equal'), plt.grid(False)
    plt.colorbar(sc1)
    plt.subplot(3, 3, 2)
    plot_vessel()
    sc2 = plt.scatter(rr, zz, c=nFx[..., ri], vmin=mFx[ri], vmax=MFx[ri], cmap=CM_R)
    plt.title('Net Fx')
    plt.axis('equal'), plt.grid(False)
    plt.colorbar(sc2)
    plt.subplot(3, 3, 3)
    plot_vessel()
    sc3 = plt.scatter(rr, zz, c=eFx[..., ri], vmin=0, vmax=max_perc_err, cmap=CM)
    plt.title('Error % Fx')
    plt.axis('equal'), plt.grid(False)  
    plt.colorbar(sc3)

    plt.subplot(3, 3, 4)
    plot_vessel()
    sc4 = plt.scatter(rr, zz, c=gBr[..., ri], vmin=mBr, vmax=MBr, cmap=CM_R)
    plt.title('LIUQUE Br')
    plt.axis('equal'), plt.grid(False)  
    plt.colorbar(sc4)
    plt.subplot(3, 3, 5)
    plot_vessel()
    sc5 = plt.scatter(rr, zz, c=nBr[..., ri], vmin=mBr, vmax=MBr, cmap=CM_R)
    plt.title('Net Br')
    plt.axis('equal'), plt.grid(False)
    plt.colorbar(sc5)
    plt.subplot(3, 3, 6)
    plot_vessel()
    sc6 = plt.scatter(rr, zz, c=eBr[..., ri], vmin=0, vmax=max_perc_err, cmap=CM)
    plt.title('Error % Br')
    plt.axis('equal'), plt.grid(False)          
    plt.colorbar(sc6)

    plt.subplot(3, 3, 7)
    plot_vessel()
    sc7 = plt.scatter(rr, zz, c=gBz[..., ri], vmin=mBz, vmax=MBz, cmap=CM_R)
    plt.title('LIUQUE Bz')
    plt.axis('equal'), plt.grid(False)
    plt.colorbar(sc7)
    plt.subplot(3, 3, 8)
    plot_vessel()
    sc8 = plt.scatter(rr, zz, c=nBz[..., ri], vmin=mBz, vmax=MBz, cmap=CM_R)
    plt.title('Net Bz')
    plt.axis('equal'), plt.grid(False)
    plt.colorbar(sc8)
    plt.subplot(3, 3, 9)
    plot_vessel()
    sc9 = plt.scatter(rr, zz, c=eBz[..., ri], vmin=0, vmax=max_perc_err, cmap=CM)
    plt.title('Error % Bz')
    plt.axis('equal'), plt.grid(False)
    plt.colorbar(sc9)

    plt.suptitle(f'Shot {sn}, t={t[ri]:.3f}', fontsize=16)
    plt.tight_layout()
    plt.show()


In [ ]:
assert False, "End of the code snippet"

In [ ]:
from matplotlib.animation import FuncAnimation
from tqdm import tqdm
from IPython.display import display, HTML
import matplotlib as mpl
mpl.rcParams['animation.embed_limit'] = 50  # value is in MB


g = loadmat('tcv_params/grid.mat')
r, z = g['r'].flatten(), g['z'].flatten()
rr, zz = np.meshgrid(r, z)
rr, zz = rr.flatten(), zz.flatten()

decimate = 1
max_perc_err = 3
fps = 30.0

for s, sn in zip(SHOT_FILES, SHOT_NAMES):
    # load data
    gt = loadmat(join(SHOTS_DIR, s))
    net = loadmat(join(SHOTS_NET_DIR, f'{sn}_net.mat'))

    gFx = gt['Fx']
    gBr = gt['Br']
    gBz = gt['Bz']
    nFx = net['Fx']
    nBr = net['Br']
    nBz = net['Bz']
    rq, zq = gt['rq'], gt['zq']
    t = gt['t'].flatten()

    # decimate
    gFx, gBr, gBz, nFx, nBr, nBz, t, rq, zq = map(lambda x: x[..., ::decimate], (gFx, gBr, gBz, nFx, nBr, nBz, t, rq, zq))

    # range
    mFx, MFx = np.min(gFx,axis=(0,1)), np.max(gFx, axis=(0, 1))

    mBr, MBr = np.min(gBr), np.max(gBr) 
    mBz, MBz = np.min(gBz), np.max(gBz) 
    rFx = MFx - mFx
    rBr = MBr - mBr
    rBz = MBz - mBz

    # precompute error percentage
    eFx = 100 * np.abs(gFx - nFx) / rFx
    eBr = 100 * np.abs(gBr - nBr) / rBr
    eBz = 100 * np.abs(gBz - nBz) / rBz

    n_frames = gFx.shape[-1]

    fig = plt.figure(figsize=(16, 9))
    axs = [plt.subplot(3, 3, i + 1) for i in range(9)]
    scatters, colorbars, plots = [], [], []

    # Initial plot setup
    def init():
        for ax in axs:
            ax.clear()
            plot_vessel(ax=ax)
            ax.axis('equal')
            ax.grid(False)
            plots.append(ax.plot(rq[:,0], zq[:,0], '--w', lw=1, alpha=0.5))  # LCFS line

        scatters.append(axs[0].scatter(rr, zz, c=gFx[..., 0], cmap=CM_R))
        axs[0].set_title('LIUQUE Fx')
        colorbars.append(plt.colorbar(scatters[-1], ax=axs[0]))

        scatters.append(axs[1].scatter(rr, zz, c=nFx[..., 0], cmap=CM_R))
        axs[1].set_title('Net Fx')
        colorbars.append(plt.colorbar(scatters[-1], ax=axs[1]))

        scatters.append(axs[2].scatter(rr, zz, c=eFx[..., 0], vmin=0, vmax=max_perc_err, cmap=CM))
        axs[2].set_title('Error % Fx')
        colorbars.append(plt.colorbar(scatters[-1], ax=axs[2]))

        scatters.append(axs[3].scatter(rr, zz, c=gBr[..., 0], vmin=mBr, vmax=MBr, cmap=CM_R))
        axs[3].set_title('LIUQUE Br')
        plt.colorbar(scatters[-1], ax=axs[3])

        scatters.append(axs[4].scatter(rr, zz, c=nBr[..., 0], vmin=mBr, vmax=MBr, cmap=CM_R))
        axs[4].set_title('Net Br')
        plt.colorbar(scatters[-1], ax=axs[4])

        scatters.append(axs[5].scatter(rr, zz, c=eBr[..., 0], vmin=0, vmax=max_perc_err, cmap=CM))
        axs[5].set_title('Error % Br')
        plt.colorbar(scatters[-1], ax=axs[5])

        scatters.append(axs[6].scatter(rr, zz, c=gBz[..., 0], vmin=mBz, vmax=MBz, cmap=CM_R))
        axs[6].set_title('LIUQUE Bz')
        plt.colorbar(scatters[-1], ax=axs[6])

        scatters.append(axs[7].scatter(rr, zz, c=nBz[..., 0], vmin=mBz, vmax=MBz, cmap=CM_R))
        axs[7].set_title('Net Bz')
        plt.colorbar(scatters[-1], ax=axs[7])

        scatters.append(axs[8].scatter(rr, zz, c=eBz[..., 0], vmin=0, vmax=max_perc_err, cmap=CM))
        axs[8].set_title('Error % Bz')
        plt.colorbar(scatters[-1], ax=axs[8])

        fig.suptitle(f'Shot {sn}, t={t[0]:.3f}/{t[-1]:.3f}', fontsize=16)

        plt.tight_layout()

        return scatters

    def update(ri):
        # --- Dynamic vmin/vmax for Fx only
        vmin_gFx = np.min(gFx[..., ri])
        vmax_gFx = np.max(gFx[..., ri])

        scatters[0].set_array(gFx[..., ri].flatten())
        scatters[0].set_clim(vmin_gFx, vmax_gFx)
        colorbars[0].update_normal(scatters[0])

        scatters[1].set_array(nFx[..., ri].flatten())
        # scatters[1].set_clim(vmin_nFx, vmax_nFx)
        scatters[1].set_clim(vmin_gFx, vmax_gFx)
        colorbars[1].update_normal(scatters[1])

        scatters[2].set_array(eFx[..., ri].flatten())
        # Keep error colorbar fixed from 0 to 1 for clarity

        scatters[3].set_array(gBr[..., ri].flatten())
        scatters[4].set_array(nBr[..., ri].flatten())
        scatters[5].set_array(eBr[..., ri].flatten())
        scatters[6].set_array(gBz[..., ri].flatten())
        scatters[7].set_array(nBz[..., ri].flatten())
        scatters[8].set_array(eBz[..., ri].flatten())

        # Update plots with LCFS
        for plot in plots:
            plot[0].set_data(rq[:, ri], zq[:, ri])

        fig.suptitle(f'Shot {sn}, t={t[ri]:.3f}/{t[-1]:.3f}', fontsize=16)
        return scatters

    # Save to MP4 with progress bar
    print(f"Rendering animation for shot {sn}...")
    with tqdm(total=n_frames, desc='Rendering frames', colour='yellow') as pbar:
        def update_with_progress(i):
            artists = update(i)
            pbar.update(1)
            return artists

        anim = FuncAnimation(fig, update_with_progress, init_func=init,
                             frames=n_frames, interval=500/fps, blit=False)
        
        # display(HTML(anim.to_html5_video()))
        # display(HTML(anim.to_jshtml()))

        anim.save(f'{SHOTS_NET_DIR}/{sn}_animation.mp4', extra_args=['-vcodec', 'libx264'])
        # import subprocess
        # subprocess.call(['xattr', '-d', 'com.apple.quarantine', f'{SHOTS_NET_DIR}/{sn}_animation.mp4']) # remove quarantine attribute for macOS

        plt.close(fig)


In [ ]:
# lets select some points along the LCFS (at fixed angles) and plot them
from utils import interp_pts
g = loadmat('tcv_params/grid.mat')
r, z = g['r'].flatten(), g['z'].flatten()
rr, zz = np.meshgrid(r, z)
rr, zz = rr.flatten(), zz.flatten()

decimate = 2
max_perc_err = 3
fps = 30.0

n_ctrl_pts = 10 # number of control points along the LCFS

nq = 129  # number of points on the LCFS

sep_idxs = np.linspace(0, nq-1, n_ctrl_pts).astype(int)
sep_angles = np.linspace(0, 2*np.pi, n_ctrl_pts, endpoint=False)
print(f'Selected {n_ctrl_pts} control points at indices: {sep_idxs}')


for s, sn in zip(SHOT_FILES, SHOT_NAMES):
    # load data
    gt = loadmat(join(SHOTS_DIR, s))
    net = loadmat(join(SHOTS_NET_DIR, f'{sn}_net.mat'))

    rq = gt['rq'] #.reshape(nq, 1, -1)  # reshape to (nq, 1, nt)
    zq = gt['zq'] #.reshape(nq, 1, -1)  # reshape to (nq, 1, nt)
    pts = np.stack([rq, zq], axis=1)

    gFx = gt['Fx']
    gBr = gt['Br']
    gBz = gt['Bz']

    t = gt['t'].flatten()

    nqFx, nqBr, nqBz = net['Fxq'], net['Brq'], net['Bzq']

    #decimate
    gFx, gBr, gBz, rq, zq, pts, t, nqFx, nqBr, nqBz = map(lambda x: x[..., ::decimate],
                                                           (gFx, gBr, gBz, rq, zq, pts, t, nqFx, nqBr, nqBz))

    nt = len(t)

    # LIUQUE values at LCFS points (interpolated from the grid at LCFS points)
    gqFx, gqBr, gqBz = np.zeros((nq, nt)), np.zeros((nq, nt)), np.zeros((nq, nt)) 
    for i in tqdm(range(nt), desc=f'Interpolating LIUQUE for shot {sn}', colour='yellow'):
        gqFx[:, i] = interp_pts(gFx[..., i], pts[..., i])
        gqBr[:, i] = interp_pts(gBr[..., i], pts[..., i])
        gqBz[:, i] = interp_pts(gBz[..., i], pts[..., i])


    # # substract mean of gqFx
    # μ = np.mean(gqFx, axis=0, keepdims=True)
    # std = np.std(gqFx, axis=0, keepdims=True)
    # print(f'Mean of gqFx: {μ.shape}, {μ}')
    # print(f'Std of gqFx: {std.shape}, {std}')
    # gqFx -= μ  # remove mean from LIUQUE Fx
    # nqFx -= μ  # remove mean from net Fx

    # net values at control points 
    gcFx, gcBr, gcBz, ncFx, ncBr, ncBz = map(lambda x: x[sep_idxs, :], (gqFx, gqBr, gqBz, nqFx, nqBr, nqBz))

    # plot
    α = 1
    color_cycle = plt.get_cmap('tab20')

    plt.figure(figsize=(16, 9))
    plt.subplot(3, 1, 1)
    for cp in range(n_ctrl_pts):
        plt.plot(t, gcFx[cp, :], label=f'GT Fx {cp}', alpha=α, color=color_cycle(cp))
        plt.plot(t, ncFx[cp, :], label=f'Net Fx {cp}', alpha=α, linestyle='--', color=color_cycle(cp))
    plt.ylabel(f'Fx')
    plt.subplot(3, 1, 2)
    for cp in range(n_ctrl_pts):
        plt.plot(t, gcBr[cp, :], label=f'GT Br {cp}', alpha=α, color=color_cycle(cp))
        plt.plot(t, ncBr[cp, :], label=f'Net Br {cp}', alpha=α, linestyle='--', color=color_cycle(cp))
    plt.ylabel(f'Br')
    plt.subplot(3, 1, 3)
    for cp in range(n_ctrl_pts):
        plt.plot(t, gcBz[cp, :], label=f'GT Bz {cp}', alpha=α, color=color_cycle(cp))
        plt.plot(t, ncBz[cp, :], label=f'Net Bz {cp}', alpha=α, linestyle='--', color=color_cycle(cp))
    plt.ylabel(f'Bz')
    plt.xlabel('Time (s)')
    plt.suptitle(f'Control points, shot {sn}', fontsize=16)
    plt.tight_layout()
    plt.savefig(f'{SHOTS_NET_DIR}/{sn}_control_points.png', bbox_inches='tight', dpi=300)
    plt.show()

    # now let's plot the errors
    α2 = 0.8
    plt.figure(figsize=(16, 9))
    plt.subplot(3, 1, 1)
    for cp in reversed(range(n_ctrl_pts)):
        zero_line = 3e-3 * cp * np.ones_like(t)
        eFx = np.abs(gcFx[cp, :] - ncFx[cp, :]) + zero_line
        # plt.plot(t, eFx, alpha=α, color=color_cycle(cp))
        plt.fill_between(t, zero_line, eFx, alpha=α2, color=color_cycle(cp))
    plt.ylabel(f'Error Fx')
    plt.subplot(3, 1, 2)
    for cp in reversed(range(n_ctrl_pts)):
        zero_line = 3e-3 * cp * np.ones_like(t)
        eBr = np.abs(gcBr[cp, :] - ncBr[cp, :]) + zero_line
        # plt.plot(t, eBr, alpha=α, color=color_cycle(cp))
        plt.fill_between(t, zero_line, eBr, alpha=α2, color=color_cycle(cp))
    plt.ylabel(f'Error Br')
    plt.subplot(3, 1, 3)
    for cp in reversed(range(n_ctrl_pts)):
        zero_line = 3e-3 * cp * np.ones_like(t)
        eBz = np.abs(gcBz[cp, :] - ncBz[cp, :]) + zero_line
        # plt.plot(t, eBz, alpha=α, color=color_cycle(cp))
        plt.fill_between(t, zero_line, eBz, alpha=α2, color=color_cycle(cp))
    plt.ylabel('Error Bz')
    plt.xlabel('Time (s)')
    plt.suptitle(f'Error on Control points, shot {sn}', fontsize=16)
    plt.tight_layout()
    plt.savefig(f'{SHOTS_NET_DIR}/{sn}_control_points_errors.png', bbox_inches='tight', dpi=300)
    plt.show()
